<a href="https://colab.research.google.com/github/jankovicsandras/plpgsql_bm25/blob/main/plpgsql_bm25_comparison_with_paradedb_pg_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

README.md

# plpgsql_bm25
## BM25 search implemented in PL/pgSQL

----
### News
 - PL/pgSQL BM25 search functions work in Postgres without any extensions / Rust

### Roadmap / TODO
 - implement other algorithms from rank_bm25, not just Okapi
 - tokenization options, stopwords
 - bm25scoressum() temp table?

----
### Contributions welcome!
The author is not a Postgres / PL/pgSQL expert, gladly accepts optimizations or constructive criticism.

----
###   Example usage:

```plpgsql
  SELECT bm25createindex( tablename, columnname );  /* tablename and columnname are TEXT types */
  SELECT * FROM bm25topk( tablename, columnname, question, k ); /* question is TEXT, k is INTGEGER */
```

----
### What is this?
 - https://en.wikipedia.org/wiki/Okapi_BM25
 - https://en.wikipedia.org/wiki/PL/pgSQL
 - https://github.com/dorianbrown/rank_bm25
 - TLDR:
    - BM25Okapi is a popular search algorithm.
    - Index building: Initially, there's a list of texts or documents called the corpus. Each document will be split to words (or tokens) with the tokenization function (the simplest is split on whitespace characters). The algorithm then builds a word-score-map ```wsmap```, where every word in the corpus is scored for every document based on their frequencies, ca. how special a word is in the corpus and how frequent in the current document.
    - Search: the question text (or query string) will be tokenized, then the search function looks up the words from ```wsmap``` and sums the scores for each document; the result is a list of scores, one for each document. The highest scoring document is the best match. The search function sorts the scores-documentIDs in descending order.
    - Adding a new document to the corpus or changing one requires rebuilding the whole BM25 index (```wsmap```), because of how the algorithm works.

----
### Repo contents
 - ```plpgsql_bm25.sql``` : PL/pgSQL functions for BM25 search
 - ```plpgsql_bm25_test.ipynb``` : Latest Jupyter notebook with comparative testing of plpgsql_bm25.sql, rank_bm25 and mybm25okapi.py.
 - ```plpgsql_bm25_dev.ipynb``` : Jupyter notebook where I develop this.
 - ```plpgsql_bm25_old_dev.ipynb``` : old
 - ```mybm25okapi.py``` : Python BM25 index builder, see also https://github.com/dorianbrown/rank_bm25

----
### Why?
Postgres has already Full Text Search and there are several extensions that implement BM25. But Full Text Search is not the same as BM25. The BM25 extensions are written in Rust, which might not be available / practical, especially in hosted environments. See Alternatives section for more info.

----
### Alternatives:

 - Postgres Full Text Search
   - https://www.postgresql.org/docs/current/textsearch.html
   - https://postgresml.org/blog/postgres-full-text-search-is-awesome


 - Rust based BM25
   - https://github.com/paradedb/paradedb/tree/dev/pg_search#overview
   - https://github.com/tensorchord/pg_bestmatch.rs


 - Postgres similarity of text using trigram matching
   - https://www.postgresql.org/docs/current/pgtrgm.html

   - NOTE: this is useful for fuzzy string matching, like spelling correction, but not query->document search solution itself.
The differing document and query text lengths will result very small relative trigram frequencies and incorrect/missing matching.

----
### Special thanks to: dorianbrown, Myon, depesz, sobel, ilmari, xiaomiao and others from #postgresql

----
### LICENSE

As https://github.com/dorianbrown/rank_bm25 has Apache-2.0 license, the derived mybm25okapi class should probably have Apache-2.0 license. The test datasets and other external code might have different licenses, please check them.

My code:

The Unlicense / PUBLIC DOMAIN

This is free and unencumbered software released into the public domain.

Anyone is free to copy, modify, publish, use, compile, sell, or distribute this software, either in source code form or as a compiled binary, for any purpose, commercial or non-commercial, and by any means.

In jurisdictions that recognize copyright laws, the author or authors of this software dedicate any and all copyright interest in the software to the public domain. We make this dedication for the benefit of the public at large and to the detriment of our heirs and successors. We intend this dedication to be an overt act of relinquishment in perpetuity of all present and future rights to this software under copyright law.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

For more information, please refer to http://unlicense.org


## installing PostgreSQL

In [1]:
! sudo apt install gnupg2 wget nano
! sudo sh -c 'echo "deb http://apt.postgresql.org/pub/repos/apt $(lsb_release -cs)-pgdg main" > /etc/apt/sources.list.d/pgdg.list'
! curl -fsSL https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo gpg --dearmor -o /etc/apt/trusted.gpg.d/postgresql.gpg
! sudo apt update
! sudo apt install postgresql-16 postgresql-contrib-16 postgresql-server-dev-16
! service postgresql start
! sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1.1).
gnupg2 is already the newest version (2.2.27-3ubuntu2.1).
Suggested packages:
  hunspell
The following NEW packages will be installed:
  nano
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 280 kB of archives.
After this operation, 881 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 nano amd64 6.2-1 [280 kB]
Fetched 280 kB in 0s (2,715 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: un

In [2]:
! export PARADEDB_TELEMETRY=false
! lsb_release -a
! sudo apt-get install -y libicu70

# https://github.com/paradedb/paradedb/releases
! wget https://github.com/paradedb/paradedb/releases/download/v0.10.2/postgresql-16-pg-search_0.10.2-1PARADEDB-jammy_amd64.deb
! dpkg -i postgresql-16-pg-search_0.10.2-1PARADEDB-jammy_amd64.deb

# https://docs.paradedb.com/deploy/pg_search
# https://github.com/paradedb/paradedb/tree/dev/pg_search

! echo "shared_preload_libraries = 'pg_search'" >> /etc/postgresql/16/main/postgresql.conf
! service postgresql stop
! sleep 10
! service postgresql start
! sudo -u postgres psql -c "CREATE EXTENSION pg_search;"

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libicu70 is already the newest version (70.1-2).
libicu70 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 54 not upgraded.
--2024-10-02 10:56:50--  https://github.com/paradedb/paradedb/releases/download/v0.10.2/postgresql-16-pg-search_0.10.2-1PARADEDB-jammy_amd64.deb
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/660776503/7312ce8f-f2ad-41f4-a375-9578db0b479e?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20241002%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20241002T105650Z&X-Amz-Expires=300&X-Amz-Signature=ba5e450d475

## init psycopg2 helper

In [3]:
! pip install psycopg2

import psycopg2


def msq( t, verbose=False ) :
  res = []
  with psycopg2.connect("dbname=postgres user=root") as conn:
    with conn.cursor() as cur:
      try :
        cur.execute(t)
        res = cur.fetchall()
        if verbose :
          for cdesc in cur.description :
            print(cdesc[0],'|',end='')
          print('')
          for r in res:
            for c in r:
              print(c,'|',end='')
            print('')
      except Exception as ex :
        print('!!!! msq() ERROR ',ex,'|',t,'|')
  return res


def iscorrectintopk(correct, res) :
  cr = correct.replace('"','\'')
  if len(res) < 1 : return 0
  for cell in res[0] :
    if str(cell).replace('"','\'') == cr : return 2
  for ri in range(1,len(res)) :
    for cell in res[ri] :
      if str(cell).replace('"','\'') == cr : return 1
  return 0


## Test dataset: Wordpress related QA from Huggingface

In [4]:

! wget https://huggingface.co/datasets/mteb/cqadupstack-wordpress/resolve/main/corpus.jsonl
! wget https://huggingface.co/datasets/mteb/cqadupstack-wordpress/resolve/main/queries.jsonl
! ls -la

import random, json

samplenum = 100

# load from jsonl
wcorpus = []
with open('corpus.jsonl') as f:
  wcstr = f.read()
  wcorpus = wcstr.split('\n')
print('len(wcorpus)',len(wcorpus))

# create sampled corpus, items, questions
sampledwcorpus = random.sample( wcorpus, samplenum )
items = []
qqs = []
for i in range(0,len(sampledwcorpus)) :
  wjs = json.loads(sampledwcorpus[i])
  #print(i,'---------------',wjs['_id'])
  #print(len(wjs['title']),wjs['title'])
  #print(len(wjs['text']),wjs['text'])
  items.append( { 'doctext': wjs['text'] } )
  qqs.append( [ wjs['title'], items[-1]['doctext'] ] )

# questions and solutions
random.shuffle(qqs)
questions = [ q[0] for q in qqs ]
questionsolutions = [ q[1] for q in qqs ]

# export to CSV for Postgres
csvfilename = 'items.csv'
with open(csvfilename,'w+') as f:
  f.write('id;doctext\n')
  for i in range(0,len(items)) :
    f.write(str((i+1))+';"'+items[i]['doctext'].replace('"','\'')+'"\n')


# Postgres creating items table by importing from CSV
msq('DROP TABLE IF EXISTS items;')
msq('CREATE TABLE items (id SERIAL, doctext TEXT);')
msq('COPY items FROM \'/content/items.csv\' DELIMITER \';\' CSV HEADER;')
#msq('SELECT * from items;')


--2024-10-02 10:57:12--  https://huggingface.co/datasets/mteb/cqadupstack-wordpress/resolve/main/corpus.jsonl
Resolving huggingface.co (huggingface.co)... 18.239.50.103, 18.239.50.80, 18.239.50.49, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.103|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/e3/a1/e3a12a6c68820b63bcd7ae09aa898026eba332004f38239070204b3b146060bc/089b11077372513eca8cc16653485aff1f232f8be18d8c6263be4b3b2bda0078?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27corpus.jsonl%3B+filename%3D%22corpus.jsonl%22%3B&Expires=1728125832&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyODEyNTgzMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2UzL2ExL2UzYTEyYTZjNjg4MjBiNjNiY2Q3YWUwOWFhODk4MDI2ZWJhMzMyMDA0ZjM4MjM5MDcwMjA0YjNiMTQ2MDYwYmMvMDg5YjExMDc3MzcyNTEzZWNhOGNjMTY2NTM0ODVhZmYxZjIzMmY4YmUxOGQ4YzYyNjNiZTRiM2IyYmRhMDA3OD9yZXNwb25zZS1jb

[]

## mybm25okapi: a refactored variant of rank_bm25 Okapi

In [5]:
"""
This is a refactored variant of rank_bm25 Okapi. https://github.com/dorianbrown/rank_bm25
This is not required for plpgsql_bm25, just here for comparison / testing.

Usage:
  - corpus and query must be tokenized already, e.g. corpus = [ ['one','two','three'], ['bla','two','two'] ]  ; query = [ 'Is', 'this', 'a', 'question?' ] ; see also mytokenize()
  - __init__(corpus) will initialize the bm25Okapi components, where self.wsmap is the most important
  - No update is possible, so if the documents change in the corpus, then __init__(corpus) must be called again (recreating all the components).
  - search with topk() or get_scores()
"""
import math


class mybm25okapi:
  def __init__(self, corpus):
    # constants
    self.debugmode = False
    self.k1 = 1.5
    self.b = 0.75
    self.epsilon = 0.25

    self.corpus_len = len(corpus)
    self.avg_doc_len = 0
    self.word_freqs = []
    self.idf = {}
    self.doc_lens = []
    word_docs_count = {}  # word -> number of documents with word
    total_word_count = 0

    for document in corpus:
      # doc lengths and total word count
      self.doc_lens.append(len(document))
      total_word_count += len(document)

      # word frequencies in this document
      frequencies = {}
      for word in document:
        if word not in frequencies:
          frequencies[word] = 0
        frequencies[word] += 1
      self.word_freqs.append(frequencies)

      # number of documents with word count
      for word, freq in frequencies.items():
        try:
          word_docs_count[word] += 1
        except KeyError:
          word_docs_count[word] = 1

    # average document length
    self.avg_doc_len = total_word_count / self.corpus_len

    if self.debugmode : print('self.corpus_len',self.corpus_len,'\nself.doc_lens',self.doc_lens,'\ntotal_word_count',total_word_count,'\nself.word_freqs',self.word_freqs,'\nself.avg_doc_len',self.avg_doc_len,'\nword_docs_count',word_docs_count)

    # precalc "half of divisor" + self.k1 * (1 - self.b + self.b * doc_lens / self.avg_doc_len)
    self.hds = [ self.k1 * ( 1-self.b + self.b*doc_len/self.avg_doc_len) for doc_len in self.doc_lens ]
    if self.debugmode : print('self.hds',self.hds)

    """
    Calculates frequencies of terms in documents and in corpus.
    This algorithm sets a floor on the idf values to eps * average_idf
    """
    # collect idf sum to calculate an average idf for epsilon value
    # collect words with negative idf to set them a special epsilon value.
    # idf can be negative if word is contained in more than half of documents
    idf_sum = 0
    negative_idfs = []
    for word, freq in word_docs_count.items():
      idf = math.log(self.corpus_len - freq + 0.5) - math.log(freq + 0.5)
      self.idf[word] = idf
      idf_sum += idf
      if idf < 0:
        negative_idfs.append(word)
      if self.debugmode : print('word',word,'self.idf[word]',self.idf[word])
    self.average_idf = idf_sum / len(self.idf)
    if self.debugmode : print('idf_sum',idf_sum,'len(self.idf)',len(self.idf),'self.average_idf',self.average_idf)
    # assign epsilon
    eps = self.epsilon * self.average_idf
    if self.debugmode : print('eps',eps)
    for word in negative_idfs:
      self.idf[word] = eps
      if self.debugmode : print('word',word,'got eps',eps)
    if self.debugmode : print('self.idf',self.idf)


    # words * documents score map
    self.wsmap = {}
    for word in self.idf :
      self.wsmap[word] = [0] * self.corpus_len
      word_freqs = [ (word_freq.get(word) or 0) for word_freq in self.word_freqs ]
      thiswordidf = (self.idf.get(word) or 0)
      if self.debugmode : print('word in self.idf',word,'thiswordidf',thiswordidf,'word_freqs',word_freqs)
      for i in range(0,self.corpus_len) :
        self.wsmap[word][i] = thiswordidf * ( word_freqs[i] * (self.k1 + 1) / ( word_freqs[i] + self.hds[i] ) ) # += replaced with =
    if self.debugmode : print('self.wsmap',self.wsmap)


  # get a list of scores for every document
  def get_scores(self, tokenizedquery):
    # zeroes list of scores
    scores = [0] * self.corpus_len
    # for each word in tokenizedquery, if word is in wsmap, lookup and add word score for every documents' scores
    for word in tokenizedquery:
      if word in self.wsmap :
        for i in range(0,self.corpus_len) :
          scores[i] += self.wsmap[word][i]
    # return scores list (not sorted)
    return scores


  def topk(self,tokenizedquery,k=None):
    docscores = self.get_scores( tokenizedquery )
    sisc = [ [i,s] for i,s in enumerate(docscores) ]
    sisc.sort(key=lambda x:x[1],reverse=True)
    if k :
      sisc = sisc[:k]
    return sisc


  # save the words*documents score map as csv for import to Postgres: COPY tablename_bm25wsmap FROM '/path-to/tablename_bm25wsmap.csv' DELIMITER ';' CSV HEADER;
  def exportwsmap(self, csvfilename) :
    with open(csvfilename,'w+') as f:
      f.write('word;vl\n')
      for word in self.wsmap :
        f.write('"'+word.replace('"','\'')+'";{'+str(self.wsmap[word]).strip()[1:-1]+'}\n')




# tokenization function
def mytokenize(s) :
  ltrimchars = ['(','[','{','<','\'','"']
  rtrimchars = ['.', '?', '!', ',', ':', ';', ')', ']', '}', '>','\'','"']
  if type(s) != str : return []
  wl = s.lower().split()
  for i,w in enumerate(wl) :
    if len(w) < 1 : continue
    si = 0
    ei = len(w)
    try :
      while si < ei and w[si] in ltrimchars : si += 1
      while ei > si and w[ei-1] in rtrimchars : ei -= 1
      wl[i] = wl[i][si:ei]
    except Exception as ex:
      print('|',w,'|',ex,'|',wl)
  wl = [ w for w in wl if len(w) > 0 ]
  return wl



In [6]:
# loading https://github.com/jankovicsandras/plpgsql_bm25

! wget https://raw.githubusercontent.com/jankovicsandras/plpgsql_bm25/refs/heads/main/plpgsql_bm25.sql

# execute to load the functions
! psql postgresql://@/postgres -f /content/plpgsql_bm25.sql


--2024-10-02 10:57:14--  https://raw.githubusercontent.com/jankovicsandras/plpgsql_bm25/refs/heads/main/plpgsql_bm25.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10804 (11K) [text/plain]
Saving to: ‘plpgsql_bm25.sql’

plpgsql_bm25.sql    100%[===================>]  10.55K  --.-KB/s    in 0s      

2024-10-02 10:57:14 (81.3 MB/s) - ‘plpgsql_bm25.sql’ saved [10804/10804]

psql:/content/plpgsql_bm25.sql:23: NOTICE:  function bm25simpletokenize() does not exist, skipping
DROP FUNCTION
CREATE FUNCTION
psql:/content/plpgsql_bm25.sql:44: NOTICE:  function count_words_in_array() does not exist, skipping
DROP FUNCTION
CREATE FUNCTION
psql:/content/plpgsql_bm25.sql:65: NOTICE:  function get_word_docs_count() does not exist, skipping
DROP FUNCTION
CREATE FUNCTIO

In [7]:

# comparative testing of plpgsql_bm25.sql, mybm25okapi and rank_bm25 BM25Okapi
# the order of results and the bm25 scores should be equal

! pip install rank_bm25

from rank_bm25 import BM25Okapi
import json, subprocess, time
import pandas as pd

# table and file names
tablename = 'items'
columnname = 'doctext'
tablename_bm25wsmap = tablename+'_bm25wsmap'
csvfilepath = '/content/'+tablename_bm25wsmap+'.csv'
k = 5
stats = {
  'plpgsql_bm25_correct':[],
  'paradedbbm25_correct':[],
  'rank_bm25_correct':[],
  'mybm25_correct':[],
  'plpgsql_bm25_speed':[],
  'paradedbbm25_speed':[],
  'rank_bm25_speed':[],
  'mybm25_speed':[],
}
verbose = False

# preparing tokenized corpus
tokenized_corpus = [ mytokenize(item[columnname]) for item in items ]

# rank_bm25 and mybm25okapi
rank_bm25_index = BM25Okapi(tokenized_corpus)
mybm25_index = mybm25okapi(tokenized_corpus)

# Postgres has already "items" table, tokenization is built-in
msq( 'SELECT bm25createindex(\''+tablename+'\',\''+columnname+'\');', verbose )

# ParadeDB pg_search
! psql postgresql://@/postgres -c "DROP SCHEMA IF EXISTS paradedbbm25 CASCADE;"
! psql postgresql://@/postgres -c "CALL paradedb.create_bm25( index_name => 'paradedbbm25', schema_name => 'public', table_name => 'items', key_field => 'id', text_fields => paradedb.field('doctext') );"
paradedbcharfilter = '\'"():-'

# Running the questions
for qi,q in enumerate(questions) :

  # tokenize and print question
  tokenizedquestion = mytokenize(q)
  if verbose :
    print('\n----Question',qi,':',q,' | Tokenized: ',tokenizedquestion)
    if questionsolutions and qi<len(questionsolutions) : print('Solution:',questionsolutions[qi])

  # plpgsql_bm25 BM25 search
  if verbose : print('plpgsql_bm25|')
  starttime = time.time()
  mres = msq( 'SELECT * FROM bm25topk( \''+tablename+'\', \''+columnname+'\', \''+q.replace("'","\'\'")+'\', '+str(k)+' );', verbose )
  stats['plpgsql_bm25_speed'].append( (time.time()-starttime) )
  stats['plpgsql_bm25_correct'].append( iscorrectintopk(questionsolutions[qi], mres) )


  # ParadeDB BM25 search
  if verbose : print('paradedbbm25|')
  # preparing query TODO
  ptokq = 'doctext:'
  for w in tokenizedquestion :
    w2 = ''.join([ c for c in w if c not in paradedbcharfilter])
    if len(w2) > 0 :
      ptokq += w2 + ' OR doctext:'
  ptokq = ptokq[:-12] + ''
  pcmdstr = 'SELECT id, doctext FROM paradedbbm25.search( \''+ptokq+'\', limit_rows => 5 );'
  # executing query
  starttime = time.time()
  mres = msq( pcmdstr, verbose )
  stats['paradedbbm25_speed'].append( (time.time()-starttime) )
  stats['paradedbbm25_correct'].append( iscorrectintopk(questionsolutions[qi], mres) )

  # rank_bm25 BM25 search
  starttime = time.time()
  doc_scores = rank_bm25_index.get_scores( tokenizedquestion )
  stats['rank_bm25_speed'].append( (time.time()-starttime) )
  bres = [ [i,s] for i,s in enumerate(doc_scores) ]
  bres.sort(key=lambda x:x[1],reverse=True)
  bres = bres[:k]
  for i in range(0,len(bres)) :
    bres[i].append( items[ bres[i][0] ][columnname] )
    bres[i][0] += 1
  stats['rank_bm25_correct'].append( iscorrectintopk(questionsolutions[qi], bres) )
  if verbose :
    print('rank_bm25  |')
    for br in bres  : print( br[0], '|', br[1], '|', br[2] ) #, items[ br[0]-1 ][columnname] )

  # mybm25okapi BM25 search
  starttime = time.time()
  bres2 = mybm25_index.topk( tokenizedquestion, k )
  stats['mybm25_speed'].append( (time.time()-starttime) )
  for i in range(0,len(bres2)) :
    bres2[i].append( items[ bres2[i][0] ][columnname] )
    bres2[i][0] += 1
  stats['mybm25_correct'].append( iscorrectintopk(questionsolutions[qi], bres2) )
  if verbose :
    print('mybm25okapi|')
    for br in bres2 : print( br[0], '|', br[1], '|', br[2] ) #items[ br[0]-1 ][columnname] )

  # print stats
  print(qi,'corrects',stats['plpgsql_bm25_correct'][-1],stats['paradedbbm25_correct'][-1],stats['rank_bm25_correct'][-1],stats['mybm25_correct'][-1],'speeds',stats['plpgsql_bm25_speed'][-1],stats['paradedbbm25_speed'][-1],stats['rank_bm25_speed'][-1],stats['mybm25_speed'][-1])


stats2 = {}
for statname in stats :
  stats2[statname+' total'] = [ sum(stats[statname]) ]

if verbose :
  df = pd.DataFrame(stats)
  #df.loc[:,'Total'] = df.sum(axis=1)
  df.style

df = pd.DataFrame(stats2)
df.style


NOTICE:  schema "paradedbbm25" does not exist, skipping
DROP SCHEMA
CALL
0 corrects 2 2 2 2 speeds 0.021229267120361328 0.07051753997802734 0.0008492469787597656 0.0003490447998046875
1 corrects 1 2 1 1 speeds 0.08496475219726562 0.0696878433227539 0.0006299018859863281 0.0002148151397705078
2 corrects 2 2 2 2 speeds 0.08501195907592773 0.07795453071594238 0.0009050369262695312 0.000308990478515625
3 corrects 2 1 2 2 speeds 0.08342623710632324 0.07024240493774414 0.0007555484771728516 0.00034618377685546875
4 corrects 1 0 1 1 speeds 0.08763933181762695 0.07454609870910645 0.0006997585296630859 0.00029206275939941406
5 corrects 2 2 2 2 speeds 0.0902245044708252 0.07355451583862305 0.0005161762237548828 0.00020837783813476562
6 corrects 2 2 2 2 speeds 0.08709359169006348 0.06986689567565918 0.0008335113525390625 0.00037360191345214844
7 corrects 2 2 2 2 speeds 0.0932924747467041 0.06858444213867188 0.0007233619689941406 0.0003800392150878906
8 corrects 2 2 2 2 speeds 0.08233904838562012 

,plpgsql_bm25_correct total,paradedbbm25_correct total,rank_bm25_correct total,mybm25_correct total,plpgsql_bm25_speed total,paradedbbm25_speed total,rank_bm25_speed total,mybm25_speed total
0,170,171,170,170,9.846076,8.510672,0.075387,0.029808
